In [17]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [18]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\huayu\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [4]:
# set up database (if needed)
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [36]:
# setting NASA url
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
# grabbing a response
response = requests.get(nasa_url)
# creating a Soup object with html parser
soup = bs(response.text, 'html.parser')

In [37]:
# grab the latest news title and paragraphs
latest_title = soup.find_all('div', class_="content_title")[0].text
latest_paragraph = soup.find_all('div', class_="rollover_description_inner")[0].text
print(latest_title)
print(latest_paragraph)



NASA's Perseverance Rover Begins Its First Science Campaign on Mars



The six-wheeled scientist is heading south to explore Jezero Crater’s lakebed in search of signs of ancient microbial life.



In [38]:
# JPL featured image url
jpl_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
jpl_space_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
browser.visit(jpl_url)

In [39]:
# finding featured image direct url
html = browser.html
img_soup = bs(html, 'html.parser')
img_src = jpl_space_url + img_soup.find_all('img', class_='headerimage fade-in')[0]['src']
img_src

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'

In [40]:
# Mars fact url
mars_url = "https://space-facts.com/mars/"
mars_facts = pd.read_html(mars_url)[0]
mars_facts

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [28]:
# setting urls and containers
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
usgs_base_url = "https://astrogeology.usgs.gov"
url_to_pages = []
img_urls = []
# turning on brower and navigate to url
browser.visit(usgs_url)
html = browser.html
usgs_soup = bs(html, 'html.parser')
# grabbing titles
titles = usgs_soup.find_all('h3')
titles = [title.text for title in titles]
# grabbing urls to each pages
for i in range(8):
    temp_url = usgs_soup.find_all('a', class_="itemLink product-item")[i]['href']
    if temp_url not in url_to_pages:
        url_to_pages.append(temp_url)
# going to each pages to grab image urls
for i in range(4):
    page_url = usgs_base_url + url_to_pages[i]
    browser.visit(page_url)
    html = browser.html
    mars_soup = bs(html, 'html.parser')
    img_urls.append(usgs_base_url + mars_soup.find('img', class_="wide-image")['src'])


In [31]:
# appending title and url to a list of dictionaries
hemi_img_urls = []
for i in range(4):
    hemi_img_urls.append({'title': titles[i], 'img_url': img_urls[i]})